In [4]:
import math as m
height= [0.2,0.4,0.8,1.6] # mm
permittivity = [ 2.2, 3.35 , 4.4 , 9.4 , 10.2 ]
freq = [0.915,2.45,4,5.8] # Ghz

input_directory = []
for h in height :
    for e in permittivity :
        for f in freq :
            temp = rf'D:\COLLEGE\LUSIP and Research\combinations\{h}\h = {h} , permitivity = {e} , freq = {f}Ghz'
            input_directory.append(temp)
input_directory

['D:\\COLLEGE\\LUSIP and Research\\combinations\\0.2\\h = 0.2 , permitivity = 2.2 , freq = 0.915Ghz',
 'D:\\COLLEGE\\LUSIP and Research\\combinations\\0.2\\h = 0.2 , permitivity = 2.2 , freq = 2.45Ghz',
 'D:\\COLLEGE\\LUSIP and Research\\combinations\\0.2\\h = 0.2 , permitivity = 2.2 , freq = 4Ghz',
 'D:\\COLLEGE\\LUSIP and Research\\combinations\\0.2\\h = 0.2 , permitivity = 2.2 , freq = 5.8Ghz',
 'D:\\COLLEGE\\LUSIP and Research\\combinations\\0.2\\h = 0.2 , permitivity = 3.35 , freq = 0.915Ghz',
 'D:\\COLLEGE\\LUSIP and Research\\combinations\\0.2\\h = 0.2 , permitivity = 3.35 , freq = 2.45Ghz',
 'D:\\COLLEGE\\LUSIP and Research\\combinations\\0.2\\h = 0.2 , permitivity = 3.35 , freq = 4Ghz',
 'D:\\COLLEGE\\LUSIP and Research\\combinations\\0.2\\h = 0.2 , permitivity = 3.35 , freq = 5.8Ghz',
 'D:\\COLLEGE\\LUSIP and Research\\combinations\\0.2\\h = 0.2 , permitivity = 4.4 , freq = 0.915Ghz',
 'D:\\COLLEGE\\LUSIP and Research\\combinations\\0.2\\h = 0.2 , permitivity = 4.4 , freq = 2

In [5]:
import re

def to_num(input_string):
    match = re.search(r'\d+(\.\d+)?', input_string)
    if match:
        return float(match.group(0))
    else:
        return None

In [ ]:
# check karega ki uski corresponding frequency range mai lie kar rhi hai ya nhi
import os
import pandas as pd
from scipy.signal import argrelextrema
import numpy as np

error = 0.3
return_loss_threshold_max = -20
return_loss_threshold_min = -40

def process_csv(file_path):
    df = pd.read_csv(file_path)
    
    min_values = []
    corresponding_frequencies = []
    cut_x = []
    cut_y = []   
    
    resonating_freq = df['Freq [GHz]'].min() + 1  #----------------------------
    if resonating_freq == 1.2 :
        resonating_freq = 0.915
    if resonating_freq == 1.415 :
        resonating_freq = 0.915
    if resonating_freq == 2.95 :
        resonating_freq = 2.45
    if resonating_freq == 4.5 :
        resonating_freq = 4
    if resonating_freq == 6.3 :
        resonating_freq = 5.8
    if resonating_freq == 5.3 :
        resonating_freq = 5.8
    # print(resonating_freq)
    
    for column in df.columns[1:]:
        local_min_indices = argrelextrema(df[column].values, np.less, order=1)[0] # will save index of minima # it returns a 2D array
        local_min_frequencies = df['Freq [GHz]'].iloc[local_min_indices]  
        local_min_values = df[column].iloc[local_min_indices]
        
        for i in range(0,len(local_min_frequencies)):
            if (resonating_freq - error <= local_min_frequencies.iloc[i] <= resonating_freq + error) and return_loss_threshold_min < local_min_values.iloc[i] < return_loss_threshold_max:   #-------------------
                min_values.append(float(local_min_values.iloc[i]))
                corresponding_frequencies.append(float(local_min_frequencies.iloc[i]))
                cut_x.append(to_num(column.split(' ')[3].split("'")[1]))
                cut_y.append(to_num(column.split(' ')[4].split("'")[1]))
            
    result_df = pd.DataFrame({
        'Cut_x': cut_x,
        'Cut_y': cut_y,
        'Min Power': min_values,
        # 'Corresponding Frequency': corresponding_frequencies
    })
    return result_df

# input_directory = r'D:\COLLEGE\LUSIP and Research\combinations\1.6\h = 1.6 , permitivity = 10.2 , freq = 0.915Ghz'
output_directory = r'D:\COLLEGE\LUSIP and Research\combinations\1.6'
output_file = os.path.join(output_directory, 'results.csv')  # Path to save the result csv file

# List to store all results
all_results = []
for path in input_directory :
    for filename in os.listdir(path):
        if filename.endswith('.csv'):
            file_path = os.path.join(path, filename)  # Get the full path of the CSV file
            result_df = process_csv(file_path)
            result_df['Height'] = float(path.split('\\')[5].split(',')[0].split(' ')[2])
            result_df['Permitivity'] = float(path.split('\\')[5].split(',')[1].split(' ')[3])
            result_df['Freq'] = to_num(path.split('\\')[5].split(',')[2].split(' ')[3])
            # result_df['Folder Name'] = path.split('\\')[5]  # Add a column to identify the file in result_df
            all_results.append(result_df)

final_results = pd.concat(all_results, ignore_index=True)  # Combine all DataFrames
final_results.to_csv(output_file, index=False)  # Save the result CSV file
print('Done')